In [ ]:
from collections import namedtuple
from collections import OrderedDict

DiskKey = namedtuple("DiskKey", ("time", "disk", "io_type"))
DiskValue = namedtuple("DiskValue", ("bytes", "io_latency_ms"))
disk_data = OrderedDict()

with open('ios', 'rt') as data:
    headers = data.readline().split(" ")
    for line in data.readlines():
        items = line.strip().split(" ")
        if len(items) != 8:
            print("Skipping line, need 8 inputs")
            continue
        time, _, _, disk, io_type, _, io_bytes, io_latency = items
        disk_data[DiskKey(float(time), disk, io_type)] = DiskValue(int(io_bytes), float(io_latency))

min_bytes, max_bytes = None, 0
min_time, max_time = None, 0
for dk, dv in disk_data.items():
    if min_bytes is None:
        min_bytes = dv.bytes
    if min_time is None:
        min_time = dk.time
        
    min_time = min(min_time, dk.time)
    max_time = max(max_time, dk.time)
    min_bytes = min(min_bytes, dv.bytes)
    max_bytes = max(max_bytes, dv.bytes)

print(min_time, max_time, min_bytes, max_bytes)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams.update({'font.size': 16})
#%matplotlib notebook

X = []
Y = []
U = []
V = []
C = []

for dk, dv in disk_data.items():
    if dv.bytes < 4096:
        continue
    X.append(dk.time)
    Y.append(dv.bytes)
    U.append(dv.io_latency_ms)
    V.append(0)
    if dk.io_type == "W":
        C.append("red")
    else:
        C.append("blue")
    
    
fig, ax = plt.subplots(figsize=(20, 10))

ax.set_title("IO Completion")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Bytes")
ax.set_yscale("log")
ax.set_xticks(np.arange(min_time, max_time, (max_time - min_time) / 20))
ax.set_yticks([2 ** i for i in range(int(np.log2(4096)), int(np.log2(max_bytes) + 1))])
ax.yaxis.set_major_formatter(matplotlib.ticker.ScalarFormatter())
# Turn on the minor TICKS, which are required for the minor GRID
ax.minorticks_on()

ax.grid(which='major', linestyle='-', linewidth='0.4', color='grey')
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='black')
ax.get_xaxis().get_major_formatter().set_useOffset(True)

q = ax.quiver(X, Y, U, V, color=C)

plt.savefig("ios.svg")
plt.show()

In [ ]:
R = []
W = []

for dk, dv in disk_data.items():
    if dv.bytes < 4096:
        continue
    if dk.io_type == "W":
        W.append(dv.io_latency_ms)
    else:
        R.append(dv.io_latency_ms)
    
fig, ax = plt.subplots(figsize=(14, 8))

ax.set_title("IO Completion")
ax.set_xlabel("Time (ms)")
ax.set_ylabel("Frequency")
ax.set_xticks(np.arange(0, 0.4, 0.025))
# Turn on the minor TICKS, which are required for the minor GRID
ax.minorticks_on()

ax.grid(which='major', linestyle='-', linewidth='0.4', color='grey')
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='black')
ax.set_xlim(0, 0.2)
ax.hist([R, W], bins="auto", color=("blue", "red"))


#plt.savefig("ios.svg")
plt.show()